## *Proposed RDH scheme through encryption*

connecting gdrive to gcollab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Divide the given Image I into non overlapping blocks of size 9x9 pixels

In [ ]:
import os
import numpy as np
from PIL import Image

# Directory containing the original images
original_images_dir = '/content/drive/MyDrive/DRDO/original image'
# Directory to save the original image in blocks
original_image_blocks_dir = '/content/drive/MyDrive/DRDO/original image blocks'

# Create the original image in blocks directory if it doesn't exist
os.makedirs(original_image_blocks_dir, exist_ok=True)

# Set the block size (e.g., 9x9)
block_size = (9, 9)


# Iterate through the original images
for filename in os.listdir(original_images_dir):
    # Open the original image
    image_path = os.path.join(original_images_dir, filename)
    original_image = Image.open(image_path)

    # Convert the image to grayscale
    grayscale_image = original_image.convert('L')

    # Split the grayscale image into blocks
    image_width, image_height = grayscale_image.size
    num_blocks_x = image_width // block_size[0]
    num_blocks_y = image_height // block_size[1]

    
    # save each image block
    for i in range(num_blocks_x):
        for j in range(num_blocks_y):
            # Extract the image block
            left = i * block_size[0]
            top = j * block_size[1]
            right = left + block_size[0]
            bottom = top + block_size[1]
            image_block = grayscale_image.crop((left, top, right, bottom))

            # Encrypt the image block
            # encrypted_block = encryption(image_block,4)

            # Save the encrypted block with a unique filename
            block_filename = f"{filename}_block_{i}_{j}.jpg"
            block_path = os.path.join(original_image_blocks_dir, block_filename)

            image_block.save(block_path)
            

    print(f"Img is converted into blocks created for {filename}")


Img is converted into blocks created for gr.png


Now original image is converted into blocks and saved into *original image block* directory

After that our task is to initialize a matrix E of size MxN with zeros
instead of doing this, we'll create another directory ie the copy of 'original image block' and in that we'll do further process of encryption

creating E as E_block by copying blocks to E_block in which we'll encrypt data

In [ ]:
import shutil

# Specify the source and destination directories
source_directory = '/content/drive/MyDrive/DRDO/original image blocks'
destination_directory = '/content/drive/MyDrive/DRDO/E_blocks'

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

# Iterate over the files in the source directory
for filename in os.listdir(source_directory):
    source_file = os.path.join(source_directory, filename)
    destination_file = os.path.join(destination_directory, filename)
    
    # Copy the file from the source directory to the destination directory
    shutil.copy2(source_file, destination_file)

print("Files copied successfully.")

Files copied successfully.


Now the next task is to find the pseudo random integer sequence R using data hiding key H. 
here we are taking value is H=11
 
So for finding pseudo random integer no of blocks(B) generated is also required
so firest we'll calculate value of B

Calculating value of B

In [ ]:
import glob

# Specify the directory path
directory_path = '/content/drive/MyDrive/DRDO/original image blocks'

# Use the glob function to search for image files in the directory
image_files = glob.glob(os.path.join(directory_path, '*.jpg'))

# Get the count of image files
num_images = len(image_files)

# Print the number of images
B= num_images


Now we'll calculate pseudo Random generated Integer (R) using data hiding key and B

In [ ]:
def generate_R(data_hiding_key, length):
    # Set the seed for the random number generator using the data hiding key
    np.random.seed(int.from_bytes(data_hiding_key.encode(), 'big'))
    
    # Generate the pseudo-random integer sequence R using the random number generator
    R = np.random.randint(low=0, high=B, size=length, dtype=np.uint8)
    
    return R


H= 11
R= generate_R('11', B)    
print(B)
print(R)

15
[ 6  6 11  3  8  2  8  6  7  8  7 12  3  4  3]


Initialize msg bit which will be further used for hiding


In [ ]:
D= [1,1,0,1,0]
l= len(D)

## Looping starts for Encryption

Encryption function

In [ ]:
# ENCRYPTION FUNCTION
def encryption(path, key):
  fin=open(path,'rb')
  imagee= fin.read()
  fin.close()
  image= bytearray(imagee)
  for index,values in enumerate(image):
    image[index]= values^ key
  fin= open(path, 'wb')
  fin.write(imagee)
  fin.close()
  print("Encryption Done...")
  # return image

Creating array of paths of original image blocks and Encrypted blocks ie E_Block
as blocks_paths and E_blocks_paths

In [ ]:
# array of paths of blocks
# import os

# Specify the directory path
blocks = '/content/drive/MyDrive/DRDO/original image blocks'

# Initialize an empty list to store the image file paths
blocks_paths = []

# Iterate over the files in the directory
for filename in os.listdir(blocks):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Construct the file path and append it to the list
        image_path = os.path.join(blocks, filename)
        blocks_paths.append(image_path)


# array of paths of E_blocks

# Specify the directory path
E_blocks = '/content/drive/MyDrive/DRDO/E_blocks'

# Initialize an empty list to store the image file paths
E_blocks_paths = []

# Iterate over the files in the directory
for filename in os.listdir(E_blocks):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Construct the file path and append it to the list
        image_path = os.path.join(E_blocks, filename)
        E_blocks_paths.append(image_path)


Replacement code to replace the image in E_block with new encrypted image

In [ ]:
# REPLACEMENT CODE 
import os
import shutil


def replacement(pos):
  # Specify the source and destination directories
  source_directory = '/content/drive/MyDrive/DRDO/original image blocks'
  destination_directory = '/content/drive/MyDrive/DRDO/E_blocks'

  # Get the list of files in the source directory
  files = os.listdir(source_directory)

  # Check if both directories contain at least 5 files
  if len(files) >= pos and len(os.listdir(destination_directory)) >= pos:
      # Get the paths of the images at the 5th position
      source_image_path = os.path.join(source_directory, files[pos-1])
      destination_image_path = os.path.join(destination_directory, os.listdir(destination_directory)[pos])

      # Replace the image
      shutil.copy2(source_image_path, destination_image_path)
      print("Image replaced successfully.")
  else:
      print("Both directories should contain at least 5 files.")


Looping...

In [ ]:
k1= 4
k2= 5
k3= 6

for x in range(0,l):
  v= R[x]
  # print(v)
  # C= image_blocks_path[v]
  C= blocks_paths[v]
  
  if(D[x]==1):
    encryption(C, k1)
    # copy the image at present at C after encryption to E_blocks[v]
    replacement(v)

  else:
    encryption(C, k2)
    # copy the image at present at C after encryption to E_blocks[v]
    replacement(v)

for x in range(l, B):
  v= R[x]
  C= blocks_paths[v]

  encryption(C, k3) 
  # copy the image at present at C after encryption to E_blocks[v]
  replacement(v) 


Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.
Encryption Done...
Image replaced successfully.


Now we have completed encryption process and also replaced it in E_block directory

in our case both original image block and E_block is encrypted

copy the image at present at C after encryption to E_blocks[v]
Defining and Encrypting rest of region  ie Ar region defination

return E
in this case we'll try to merge all the images of E_block and make a complete one Encrypted image

In [ ]:
# Merge all blocks of E_block to make a image
import cv2
import numpy as np

# List of block paths
block_paths = E_blocks_paths
# Create an empty list to store the image blocks
image_blocks = []

# Load and append each image block to the list
for block_path in block_paths:
    image_block = cv2.imread(block_path)
    image_blocks.append(image_block)

# Combine the image blocks into a single image
merged_image = np.concatenate(image_blocks, axis=1)

# # Display or save the merged image
# cv2.imshow("Merged Image", merged_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# merged_image

Now we have got final merged image of all the encrypted block which is denoted by **merged_image**. now this merged image is sent to reciever and then further decryption process will occur. Before that SVM model will be required which will be used to classify if the image block is encrypted or not

## SVM Model training

So to train model we need data set of both encrypted and non encrypted images. 
for that we'll collect 10-15 grayscale images for non encrypted image dataset and then encrypt it to make encrypted dataset.  
we'll use the same encryption technique that we use in encryption process

**Creating Encrypted image dataset**

In [ ]:
import os
import numpy as np
from PIL import Image

# Function to perform pixel scrambling encryption on an image block
def encrypt_image_block(image_block):
    # Convert the image block to a numpy array
    block_array = np.array(image_block)

    # Get the shape of the block
    height, width = block_array.shape[:2]

    # Flatten the block into a 1D array
    flattened_block = block_array.flatten()

    # Randomly shuffle the pixel values
    np.random.shuffle(flattened_block)

    # Reshape the flattened block back to its original shape
    encrypted_block = flattened_block.reshape((height, width))

    # Convert the encrypted block back to PIL Image
    encrypted_image_block = Image.fromarray(encrypted_block)

    return encrypted_image_block


# Directory containing the original images
original_images_dir = '/content/drive/MyDrive/DRDO/normal image'

# Directory to save the encrypted image blocks
encrypted_images_dir = '/content/drive/MyDrive/DRDO/encrypted image'

# Create the encrypted blocks directory if it doesn't exist
os.makedirs(encrypted_images_dir, exist_ok=True)

# Set the block size (e.g., 9x9)
block_size = (9, 9)

# Iterate through the original images
for filename in os.listdir(original_images_dir):
    # Open the original image
    image_path = os.path.join(original_images_dir, filename)
    original_image = Image.open(image_path)

    # Convert the image to grayscale
    grayscale_image = original_image.convert('L')

    # Split the grayscale image into blocks
    image_width, image_height = grayscale_image.size
    num_blocks_x = image_width // block_size[0]
    num_blocks_y = image_height // block_size[1]


    # Encrypt and save each image block
    for i in range(num_blocks_x):
        for j in range(num_blocks_y):
            # Extract the image block
            left = i * block_size[0]
            top = j * block_size[1]
            right = left + block_size[0]
            bottom = top + block_size[1]
            image_block = grayscale_image.crop((left, top, right, bottom))

            # Encrypt the image block
            encrypted_block = encrypt_image_block(image_block)

            # Save the encrypted block with a unique filename
            block_filename = f"{filename}_block_{i}_{j}.jpg"
            block_path = os.path.join(encrypted_images_dir, block_filename)
            encrypted_block.save(block_path)
            

    print(f"Encrypted blocks created for {filename}")


Encrypted blocks created for 2.jpg
Encrypted blocks created for 3.jpg
Encrypted blocks created for 4.jpg
Encrypted blocks created for 5.jpg
Encrypted blocks created for 1.jpg
Encrypted blocks created for 6.jpg
Encrypted blocks created for 7.jpg
Encrypted blocks created for 8.jpg
Encrypted blocks created for 9.jpg
Encrypted blocks created for 10.jpg


# Creating non encrypted Image blocks

In [ ]:
import os
from PIL import Image

# Directory containing the original images
original_images_dir = '/content/drive/MyDrive/DRDO//normal image'

# Directory to save the non-encrypted image blocks
non_encrypted_blocks_dir = '/content/drive/MyDrive/DRDO/non encrypted image blocks'

# Create the non-encrypted blocks directory if it doesn't exist
os.makedirs(non_encrypted_blocks_dir, exist_ok=True)

# Set the block size (e.g., 32x32)
block_size = (9, 9)

# Iterate through the original images
for filename in os.listdir(original_images_dir):
    # Open the original image
    image_path = os.path.join(original_images_dir, filename)
    original_image = Image.open(image_path)

    # Convert the image to grayscale
    grayscale_image = original_image.convert('L')

    # Split the grayscale image into blocks
    image_width, image_height = grayscale_image.size
    num_blocks_x = image_width // block_size[0]
    num_blocks_y = image_height // block_size[1]

    # Extract and save each image block
    for i in range(num_blocks_x):
        for j in range(num_blocks_y):
            # Extract the image block
            left = i * block_size[0]
            top = j * block_size[1]
            right = left + block_size[0]
            bottom = top + block_size[1]
            image_block = grayscale_image.crop((left, top, right, bottom))

            # Save the image block with a unique filename
            block_filename = f"{filename}_block_{i}_{j}.png"
            block_path = os.path.join(non_encrypted_blocks_dir, block_filename)
            image_block.save(block_path)

    print(f"Non-encrypted blocks created for {filename}")


Non-encrypted blocks created for 2.jpg
Non-encrypted blocks created for 3.jpg


FileNotFoundError: ignored

## SVM Model

In [ ]:
# Function to extract features from an image block
def extract_features(image_block):
    # image_block = np.array(img_block)
    features = []

    # Standard deviation (SD)
    sd = np.std(image_block)
    features.append(sd)

    # Quantization
    quantized_block = np.floor_divide(image_block, 64).astype(np.uint8)

    # Peak in the histogram after quantization (PK)
    hist = cv2.calcHist([quantized_block], [0], None, [64], [0, 64])
    pk = np.max(hist)
    features.append(pk)

    # Maximum vote from histogram bins (VH)
    hist_bins = cv2.calcHist([quantized_block], [0], None, [32], [0, 64])
    vh = np.max(hist_bins)
    features.append(vh)

    # Entropy (EP)
    hist_normalized = hist / np.sum(hist)
    entropy = -np.sum(hist_normalized * np.log2(hist_normalized + 1e-7))
    features.append(entropy)

    # Correlation between adjacent pixels (CP)
    # cp = np.mean(np.abs(image_block[:-1, :-1] - image_block[1:, 1:]))
    # features.append(cp)

    # Smoothness measure (SM)
    # sm = np.mean(np.abs(image_block[1:-1, 1:-1] - (image_block[:-2, 1:-1] + image_block[2:, 1:-1] + image_block[1:-1, :-2] + image_block[1:-1, 2:])/4))
    # features.append(sm)

    return features

In [ ]:
import cv2
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



# Load encrypted and non-encrypted image blocks
import os
from PIL import Image

def load_image_blocks(directory):
    image_blocks = []
    for filename in os.listdir(directory):
        # Create the file path
        file_path = os.path.join(directory, filename)
        # Open the image block using PIL
        image_block = Image.open(file_path)
        # Convert the image block to numpy array or perform any other required processing
        # ...
        # Add the image block to the list
        image_blocks.append(image_block)
    return image_blocks

# Specify the directory paths for encrypted and non-encrypted image blocks
encrypted_blocks_dir = '/content/drive/MyDrive/DRDO/original image blocks'
non_encrypted_blocks_dir = '/content/drive/MyDrive/DRDO/non encrypted image blocks'

# Load encrypted image blocks
encrypted_image_blocks = load_image_blocks(encrypted_blocks_dir)

# Load non-encrypted image blocks
non_encrypted_image_blocks = load_image_blocks(non_encrypted_blocks_dir)

# Now you have the encrypted and non-encrypted image blocks loaded into memory,
# and you can use them for further processing or analysis


# reeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee


# Load encrypted and non-encrypted image blocks
# encrypted_blocks = np.load('/content/drive/MyDrive/img classification/grayscale images/encrypted_img_blocks/1.jpg_block_0_0.jpg')
# non_encrypted_blocks = np.load('/content/drive/MyDrive/img classification/grayscale images/non encrypted image blocks')

# Extract features and create feature vectors
X_encrypted = np.array([extract_features(block) for block in encrypted_image_blocks])
X_non_encrypted = np.array([extract_features(block) for block in non_encrypted_image_blocks])

# Create labels
y_encrypted = np.ones(len(encrypted_image_blocks))
y_non_encrypted = np.zeros(len(non_encrypted_image_blocks))

# Combine encrypted and non-encrypted feature vectors and labels
X = np.vstack((X_encrypted, X_non_encrypted))
y = np.hstack((y_encrypted, y_non_encrypted))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM model
svm_model = svm.SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Predict on the testing set
y_pred = svm_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Save the trained SVM model for future use
import pickle
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(svm_model, f)


Accuracy: 0.9985433357611071


## Data extraction and image recovery

Divide the image E into non-overlapping blocks of size S × S pixels. in our case E is merged_image

1st save merged_image in new directory called Final_encrypted_image


In [ ]:
output_directory= '/content/drive/MyDrive/DRDO/AfterEncryption/Final_encrypted_image'

output_path = os.path.join(output_directory, "merged_image.jpg")
cv2.imwrite(output_path, merged_image)


Divide the merged image into blocks and will save it into new directory called "Final Encrypted Image Blocks"

In [ ]:
import os
import numpy as np
from PIL import Image

# Directory containing the original images
AfterEncryption_images_dir = '/content/drive/MyDrive/DRDO/AfterEncryption/Final_encrypted_image'
# Directory to save the original image in blocks
AfterEncryption_images_blocks_dir = '/content/drive/MyDrive/DRDO/AfterEncryption/Final_encrypted_image_blocks'

# Create the original image in blocks directory if it doesn't exist
os.makedirs(AfterEncryption_images_blocks_dir, exist_ok=True)

# Set the block size (e.g., 9x9)
block_size = (9, 9)


# Iterate through the original images
for filename in os.listdir(AfterEncryption_images_dir):
    # Open the original image
    image_path = os.path.join(AfterEncryption_images_dir, filename)
    original_image = Image.open(image_path)

    # Convert the image to grayscale
    grayscale_image = original_image.convert('L')

    # Split the grayscale image into blocks
    image_width, image_height = grayscale_image.size
    num_blocks_x = image_width // block_size[0]
    num_blocks_y = image_height // block_size[1]

    
    # save each image block
    for i in range(num_blocks_x):
        for j in range(num_blocks_y):
            # Extract the image block
            left = i * block_size[0]
            top = j * block_size[1]
            right = left + block_size[0]
            bottom = top + block_size[1]
            image_block = grayscale_image.crop((left, top, right, bottom))

            # Encrypt the image block
            # encrypted_block = encryption(image_block,4)

            # Save the encrypted block with a unique filename
            block_filename = f"{filename}_block_{i}_{j}.jpg"
            block_path = os.path.join(original_image_blocks_dir, block_filename)

            image_block.save(block_path)
            

    print(f"Img is converted into blocks created for {filename}")


Find pseudo-random integer sequence R using data hiding key H

In [ ]:
# B= len(blocks)
def generate_R(data_hiding_key, length):
    # Set the seed for the random number generator using the data hiding key
    np.random.seed(int.from_bytes(data_hiding_key.encode(), 'big'))
    
    # Generate the pseudo-random integer sequence R using the random number generator
    R = np.random.randint(low=0, high=15, size=length, dtype=np.uint8)
    
    return R
H= 11
# calculate the value of B ie number of blocks in merged_image
B= len(merged_image)
R= generate_R('11', B)    
print(B)
print(R)

In [ ]:
# M,N= merged_image.shape

B1= math.floor(M/S) * math.floor(N/S)

## Decryption

In [ ]:
def decryption(path, key):

  try:
    # path=path
    # key = 4

    fin= open(path,'rb')
    imagee= fin.read()
    fin.close()
    imagee= bytearray(imagee)
    for index,values in enumerate(imagee):
      imagee[index]= values^ key

    fin= open(path, 'wb')
    fin.write(imagee)
    fin.close() 

    print("Decryption done...")
    # return 

  except Exception:
    print("Error caught: ", Exception._name_)  


creating E_block_paths1 and 2 

In [ ]:
# array of paths of E_blocks
import os

# Specify the directory path
E_blocks1 = '/content/drive/MyDrive/DRDO/forDecryption/E_blocks1'

# Initialize an empty list to store the image file paths
E_blocks_paths1 = []

# Iterate over the files in the directory
for filename in os.listdir(E_blocks1):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Construct the file path and append it to the list
        image_path = os.path.join(E_blocks1, filename)
        E_blocks_paths1.append(image_path)


In [ ]:
# array of paths of E_blocks2
import os

# Specify the directory path
E_blocks2 = '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2'

# Initialize an empty list to store the image file paths
E_blocks_paths2 = []

# Iterate over the files in the directory
for filename in os.listdir(E_blocks2):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Construct the file path and append it to the list
        image_path = os.path.join(E_blocks2, filename)
        E_blocks_paths2.append(image_path)


In [ ]:
E_blocks_paths2

['/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_1_1.jpg',
 '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_0_2.jpg',
 '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_1_0.jpg',
 '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_0_0.jpg',
 '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_0_1.jpg',
 '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_2_0.jpg',
 '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_4_2.jpg',
 '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_3_0.jpg',
 '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_4_0.jpg',
 '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_4_1.jpg',
 '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_3_2.jpg',
 '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_1_2.jpg',
 '/content/drive/MyDrive/DRDO/forDecryption/E_blocks2/gr.png_block_2_1.jpg',

In [ ]:
for x in range(0, B):
  v = R[x] 
  # C = E[v]
  # E_blocks_paths
  Cc1= E_blocks_paths[v]
  Cc2= E_blocks_paths1[v]
  Cc3= E_blocks_paths2[v]

  k = 1
  # C= C.tolist()
  C1 = decryption(Cc1, k1)
  C2 = decryption(Cc2, k2) 
  C3 = decryption(Cc3, k3)
  # Classify(C1, C2, C3)  ie. predict which one of these 3 is normal after decryption
  c1 = svm_model.predict(C1)
  c2= svm_model.predict(C2)
  c3= svm_model.predict(C3)
  if(c1==0)
    G[v]= C1
    W[k]= 0
    k= k+1
  elif(c2==0):
    G[v]= C2
    W[k]= 1
    k= k+1
  else:
    G[v]= C3

return k    


Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...
Decryption done...


Now Decryption is done